# Spark Arrow compression test 
In this section, we use spark and arrow to output parquet files with different compression algo.

In [1]:
from pyspark.sql import SparkSession,DataFrame
import os
import numpy as np
from pyspark.sql import functions as f
from pyspark.sql.types import *
import io
import time
from pyspark.sql import Row


In [18]:
local=False
# spark.rpc.message.maxSize if for write large csv file. The default value is 128, here we set it to 1024
if local:
    spark = SparkSession \
    .builder.master("local[4]") \
    .appName("SparkArrowCompression") \
    .getOrCreate()
else: 
    spark = SparkSession \
    .builder.master("k8s://https://kubernetes.default.svc:443") \
    .appName("SparkArrowCompression") \
    .config("spark.kubernetes.container.image", "inseefrlab/jupyter-datascience:master") \
    .config("spark.kubernetes.authenticate.driver.serviceAccountName", os.environ['KUBERNETES_SERVICE_ACCOUNT']) \
    .config("spark.executor.instances", "4") \
    .config("spark.executor.memory","8g") \
    .config("spark.kubernetes.namespace", os.environ['KUBERNETES_NAMESPACE']) \
    .getOrCreate()

In [13]:
spark.sparkContext.stop()

In [2]:
! kubectl get pods

I0924 09:19:59.144597    1725 request.go:655] Throttling request took 1.164686495s, request: GET:https://kubernetes.default/apis/authorization.k8s.io/v1?timeout=32s
NAME                               READY   STATUS    RESTARTS   AGE
flume-test-agent-df8c5b944-vtjbx   1/1     Running   0          4d19h
jupyter-324928-7b4cdf67dd-tk99l    1/1     Running   0          24h
jupyter-542018-6d9c59bc68-vrzh4    1/1     Running   0          97m
kafka-server-0                     1/1     Running   0          4d20h
kafka-server-1                     1/1     Running   0          4d19h
kafka-server-2                     1/1     Running   0          4d21h
kafka-server-zookeeper-0           1/1     Running   0          4d19h


In [16]:
! kubectl get pods | grep Error | awk '{print $1}' | xargs kubectl delete pods

I0924 07:57:12.365932     685 request.go:655] Throttling request took 1.177662055s, request: GET:https://kubernetes.default/apis/apiextensions.k8s.io/v1?timeout=32s
I0924 07:57:15.715832     719 request.go:655] Throttling request took 1.179433162s, request: GET:https://kubernetes.default/apis/networking.k8s.io/v1beta1?timeout=32s
pod "sparkarrowcompression-03c6b67c11df10d9-exec-1" deleted
pod "sparkarrowcompression-03c6b67c11df10d9-exec-2" deleted
pod "sparkarrowcompression-03c6b67c11df10d9-exec-3" deleted
pod "sparkarrowcompression-03c6b67c11df10d9-exec-4" deleted


In [54]:
parquet_input_path = "s3a://pengfei/diffusion/data_format/sf_fire/parquet/raw"
compress_output_path = "s3a://pengfei/diffusion/data_format/sf_fire/parquet"
output_path="s3a://pengfei/diffusion/data_format/spark_netflix/"
csv_input_path="s3a://pengfei/diffusion/data_format/ny_taxis/csv"

In [52]:
fireSchema = StructType([StructField('CallNumber', IntegerType(), True),
                     StructField('UnitID', StringType(), True),
                     StructField('IncidentNumber', IntegerType(), True),
                     StructField('CallType', StringType(), True),                  
                     StructField('CallDate', StringType(), True),       
                     StructField('WatchDate', StringType(), True),       
                     StructField('ReceivedDtTm', StringType(), True),       
                     StructField('EntryDtTm', StringType(), True),       
                     StructField('DispatchDtTm', StringType(), True),       
                     StructField('ResponseDtTm', StringType(), True),       
                     StructField('OnSceneDtTm', StringType(), True),       
                     StructField('TransportDtTm', StringType(), True),                  
                     StructField('HospitalDtTm', StringType(), True),       
                     StructField('CallFinalDisposition', StringType(), True),       
                     StructField('AvailableDtTm', StringType(), True),       
                     StructField('Address', StringType(), True),       
                     StructField('City', StringType(), True),       
                     StructField('ZipcodeofIncident', IntegerType(), True),       
                     StructField('Battalion', StringType(), True),                 
                     StructField('StationArea', StringType(), True),       
                     StructField('Box', StringType(), True),       
                     StructField('OriginalPriority', StringType(), True),       
                     StructField('Priority', StringType(), True),       
                     StructField('FinalPriority', IntegerType(), True),       
                     StructField('ALSUnit', BooleanType(), True),       
                     StructField('CallTypeGroup', StringType(), True),
                     StructField('NumberofAlarms', IntegerType(), True),
                     StructField('UnitType', StringType(), True),
                     StructField('Unitsequenceincalldispatch', IntegerType(), True),
                     StructField('FirePreventionDistrict', StringType(), True),
                     StructField('SupervisorDistrict', StringType(), True),
                     StructField('NeighborhoodDistrict', StringType(), True),
                     StructField('Location', StringType(), True),
                     StructField('RowID', StringType(), True)])
data_path="s3a://pengfei/diffusion/data_format/Fire_Department_Calls_for_Service.csv"
df = spark.read.options(delimiter=',').option("header","true").schema(fireSchema).csv(data_path)
print(df.columns)
df.show(1)
df.printSchema()



['CallNumber', 'UnitID', 'IncidentNumber', 'CallType', 'CallDate', 'WatchDate', 'ReceivedDtTm', 'EntryDtTm', 'DispatchDtTm', 'ResponseDtTm', 'OnSceneDtTm', 'TransportDtTm', 'HospitalDtTm', 'CallFinalDisposition', 'AvailableDtTm', 'Address', 'City', 'ZipcodeofIncident', 'Battalion', 'StationArea', 'Box', 'OriginalPriority', 'Priority', 'FinalPriority', 'ALSUnit', 'CallTypeGroup', 'NumberofAlarms', 'UnitType', 'Unitsequenceincalldispatch', 'FirePreventionDistrict', 'SupervisorDistrict', 'NeighborhoodDistrict', 'Location', 'RowID']
+----------+------+--------------+--------+----------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+------------+--------------------+--------------------+--------------------+-------------+-----------------+---------+-----------+----+----------------+--------+-------------+-------+-------------+--------------+--------+--------------------------+----------------------+----------

In [29]:
def check_spark_parquet_read_time(path:str)->DataFrame:
    t1=time.time()
    df=spark.read.parquet(path)
    print(f"data frame has {df.count()} rows, {len(df.columns)} columns")
    t2=time.time()
    print(f"Spark read above data frame in parquet format, and spents: {t2 - t1} s")
    return df

# read parquet generated by arrow    
df=check_spark_parquet_read_time(parquet_input_path)

df.printSchema()

# read parquet generated by spark
# check_spark_parquet_read_time("s3a://pengfei/diffusion/data_format/netflix.parquet")

data frame has 5500520 rows, 34 columns
Spark read above data frame in parquet format, and spents: 1.9366462230682373 s
root
 |-- CallNumber: integer (nullable = true)
 |-- UnitID: string (nullable = true)
 |-- IncidentNumber: integer (nullable = true)
 |-- CallType: string (nullable = true)
 |-- CallDate: string (nullable = true)
 |-- WatchDate: string (nullable = true)
 |-- ReceivedDtTm: string (nullable = true)
 |-- EntryDtTm: string (nullable = true)
 |-- DispatchDtTm: string (nullable = true)
 |-- ResponseDtTm: string (nullable = true)
 |-- OnSceneDtTm: string (nullable = true)
 |-- TransportDtTm: string (nullable = true)
 |-- HospitalDtTm: string (nullable = true)
 |-- CallFinalDisposition: string (nullable = true)
 |-- AvailableDtTm: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- ZipcodeofIncident: integer (nullable = true)
 |-- Battalion: string (nullable = true)
 |-- StationArea: string (nullable = true)
 |-- Box: strin

In [6]:
def check_spark_csv_write_time(df:DataFrame,path:str):
    t1=time.time()
    df.coalesce(1).write.option("header","true").csv(path)
    print(f"data frame has {df.count()} rows, {len(df.columns)} columns")
    t2=time.time()
    print(f"Spark read time spents: {t2 - t1} s")

# check_spark_csv_write_time(df,f"{csv_input_path}/2011_2012")   

def check_spark_csv_read_time(path):
    t1=time.time()
    df=spark.read.csv(path)
    print(f"data frame has {df.count()} rows, {len(df.columns)} columns")
    t2=time.time()
    print(f"Spark read time spents: {t2 - t1} s")
    return df

# df_fire=check_spark_read_csv_time(csv_example)


In [7]:
def check_spark_parquet_write_time(df,path,partition_number,compression_algo):
    t1=time.time()
    df.coalesce(partition_number).write \
    .option("parquet.compression",compression_algo) \
    .parquet(path) 
    t2=time.time()
    print(f"Spark write parquet with {compression_algo} compression, it spents : {t2 - t1} s")
 

# 1. Spark compression example

# 1.1 Spark Compress with gzip

In [55]:
# Spark write parquet with gzip compression ny 2009, it spents : 327.7600781917572 s

comp_algo="gzip"
check_spark_parquet_write_time(df,f"{compress_output_path}/spark_sf_fire_{comp_algo}",4,comp_algo)


Spark write parquet with gzip compression, it spents : 74.42315220832825 s


In [25]:
! mc ls --summarize s3/pengfei/diffusion/data_format/sf_fire/parquet/spark_sf_fire_gzip | grep "Total Size"

Total Size: 355 MiB


## 1.2 Spark compress with snappy

In [56]:
#Spark write parquet with snappy compression ny 2009, it spents : 210.80829095840454 s

comp_algo="snappy"
check_spark_parquet_write_time(df,f"{compress_output_path}/spark_sf_fire_{comp_algo}",4,comp_algo)


Spark write parquet with snappy compression, it spents : 50.436259031295776 s


In [57]:
! mc ls --summarize s3/pengfei/diffusion/data_format/sf_fire/parquet/spark_sf_fire_snappy | grep "Total Size"

Total Size: 552 MiB


## 1.3 Spark Compress with lz4
missing lz4 dependencies, but the doc says it's supported by default 
https://spark.apache.org/docs/latest/sql-data-sources-parquet.html

In [ ]:
comp_algo="lz4"
check_spark_parquet_write_time(df,f"{compress_output_path}/2009_{comp_algo}",4,comp_algo)

## 1.4 Spark compress with lzo
missing lzo dependencies, but the doc says it's supported by default 
https://spark.apache.org/docs/latest/sql-data-sources-parquet.html

In [ ]:
comp_algo="lzo"
check_spark_parquet_write_time(df,f"{compress_output_path}/2009_{comp_algo}",4,comp_algo)

## 1.5 Spark compress with brotli
doc says it's not supported by default, so missing brotli dependencies is normal

In [ ]:
comp_algo="brotli"
check_spark_parquet_write_time(df,f"{compress_output_path}/2009_{comp_algo}",4,comp_algo)

## 1.6 Spark compress with zstd
doc says it's not supported by default, so missing zstd dependencies is normal

In [ ]:
# zstd is not supported by default
comp_algo="zstd"
check_spark_parquet_write_time(df,f"{compress_output_path}/2019_{comp_algo}",4,comp_algo)

## Pyarrow writes parquet with compression


In [2]:
import pathlib
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import s3fs
import os
import time

In [3]:
# This function reads a parquet data set (partitioned partque files) from s3, and returns an arrow table
def read_parquet_from_s3(endpoint: str, bucket_name, path):
    url = f"https://{endpoint}"
    fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': url})
    file_uri = f"{bucket_name}/{path}"
    str_info = fs.info(file_uri)
    print(f"input file metadata: {str_info}")
    dataset = pq.ParquetDataset(file_uri, filesystem=fs, metadata_nthreads=8)
    table = dataset.read()
    return table

# check read time
def check_arrow_read_time(endpoint, bucket, path):
    t1 = time.time()
    arrow_table=read_parquet_from_s3(endpoint, bucket, path)
    get_shape(arrow_table)
    t2 = time.time()
    print(f"Arrow read time spents: {t2 - t1} s")
    return arrow_table
    
# This function reads an arrow table, convert it to a pandas dataframe, then return the shape of the dataframe. 
def get_shape(table):
    df = table.to_pandas()
    print(f"shape of the data set: {df.shape}")

In [4]:
endpoint = os.environ['AWS_S3_ENDPOINT']
bucket = "pengfei"
# don't add / after raw, it will raise error
input_path = "diffusion/data_format/sf_fire/parquet/raw"

In [5]:
arrow_table=check_arrow_read_time(endpoint,bucket, input_path)

input file metadata: {'Key': 'pengfei/diffusion/data_format/sf_fire/parquet/raw', 'name': 'pengfei/diffusion/data_format/sf_fire/parquet/raw', 'type': 'directory', 'Size': 0, 'size': 0, 'StorageClass': 'DIRECTORY'}
shape of the data set: (5500519, 34)
Arrow read time spents: 27.993977308273315 s


In [6]:
pdf=arrow_table.to_pandas()
print(pdf.head(2))

   CallNumber UnitID  IncidentNumber CallType    CallDate   WatchDate  \
0   210391607    E19        21017645   Alarms  02/08/2021  02/08/2021   
1   210391164    T04        21017596   Alarms  02/08/2021  02/08/2021   

             ReceivedDtTm               EntryDtTm            DispatchDtTm  \
0  02/08/2021 01:00:14 PM  02/08/2021 01:01:36 PM  02/08/2021 01:01:40 PM   
1  02/08/2021 10:54:56 AM  02/08/2021 10:56:50 AM  02/08/2021 10:56:57 AM   

             ResponseDtTm  ... ALSUnit CallTypeGroup NumberofAlarms UnitType  \
0  02/08/2021 01:03:21 PM  ...    True         Alarm              1   ENGINE   
1  02/08/2021 10:57:07 AM  ...   False         Alarm              1    TRUCK   

  Unitsequenceincalldispatch FirePreventionDistrict SupervisorDistrict  \
0                        1.0                      8                  7   
1                        1.0                      3                  6   

   NeighborhoodDistrict       Location  \
0             Lakeshore  210391607-E19   


In [7]:
# This function write an arrow table to s3 as parquet files, you can specify a compression type
# compression (str or dict) – Specify the compression codec, either on a general basis or per-column. 
# Valid values: {‘NONE’, ‘SNAPPY’, ‘GZIP’, ‘BROTLI’, ‘LZ4’, ‘ZSTD’}.
# default is snappy.

def write_parquet_as_partitioned_dataset(table, endpoint, bucket_name, path, partition_cols=None, compression="SNAPPY"):
    url = f"https://{endpoint}"
    fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': url})
    file_uri = f"{bucket_name}/{path}"
    pq.write_to_dataset(table, root_path=file_uri, partition_cols=partition_cols, filesystem=fs, compression=compression)
    
# check write time
def check_write_time(table, endpoint, bucket_name, path, partition_cols=None, compression="SNAPPY"):
    t1=time.time()
    write_parquet_as_partitioned_dataset(table, endpoint, bucket_name, path, partition_cols,compression=compression)
    t2=time.time()
    print(f"Arrow write time spents: {t2 - t1} s")

In [11]:
output_path=f"diffusion/data_format/sf_fire/parquet/arrow_sf_fire_snappy"
check_write_time(arrow_table,endpoint,bucket,output_path)

Arrow write time spents: 37.47578525543213 s


In [10]:
! mc ls --summarize s3/pengfei/diffusion/data_format/sf_fire/parquet/arrow_sf_fire_snappy | grep "Total Size"

Total Size: 596 MiB


In [12]:
output_path="diffusion/data_format/sf_fire/parquet/arrow_sf_fire_zstd"
check_write_time(arrow_table,endpoint,bucket,output_path,compression="ZSTD")

Arrow write time spents: 29.63837957382202 s


In [13]:
! mc ls --summarize s3/pengfei/diffusion/data_format/sf_fire/parquet/arrow_sf_fire_zstd | grep "Total Size"

Total Size: 385 MiB


In [14]:
output_path="diffusion/data_format/sf_fire/parquet/arrow_sf_fire_gzip"
check_write_time(arrow_table,endpoint,bucket,output_path,compression="GZIP")

Arrow write time spents: 138.10498523712158 s


In [15]:
! mc ls --summarize s3/pengfei/diffusion/data_format/sf_fire/parquet/arrow_sf_fire_gzip | grep "Total Size"

Total Size: 372 MiB


In [16]:
output_path="diffusion/data_format/sf_fire/parquet/arrow_sf_fire_lz4"
check_write_time(arrow_table,endpoint,bucket,output_path,compression="LZ4")

Arrow write time spents: 38.89105200767517 s


In [17]:
! mc ls --summarize s3/pengfei/diffusion/data_format/sf_fire/parquet/arrow_sf_fire_lz4 | grep "Total Size"

Total Size: 589 MiB


In [8]:
output_path="diffusion/data_format/sf_fire/parquet/arrow_sf_fire_brotli"
check_write_time(arrow_table,endpoint,bucket,output_path,compression="BROTLI")

Arrow write time spents: 163.5519528388977 s


In [9]:
! mc ls --summarize s3/pengfei/diffusion/data_format/sf_fire/parquet/arrow_sf_fire_brotli | grep "Total Size"

Total Size: 332 MiB


In [10]:
output_path="diffusion/data_format/sf_fire/parquet/arrow_sf_fire_none"
check_write_time(arrow_table,endpoint,bucket,output_path,compression="NONE")

Arrow write time spents: 72.52444410324097 s


In [11]:
! mc ls --summarize s3/pengfei/diffusion/data_format/sf_fire/parquet/arrow_sf_fire_none | grep "Total Size"

Total Size: 1.3 GiB


# compression and use dictionary encoding by column
arrow allows us to specify compression and dictionary codec per column
spark does not, spark can only specify a global compression and dicionary codec.

pq.write_table(table, where, compression={'foo': 'snappy', 'bar': 'gzip'}, use_dictionary=['foo', 'bar'])

# to test if we have a mix compression, spark can read correctly or not.

Write a mixed compression

In [24]:
url = f"https://{endpoint}"
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': url})
path="diffusion/data_format/sf_fire/parquet/arrow_mix_compression"
file_uri = f"{bucket}/{path}"
pq.write_table(arrow_table, file_uri, filesystem=fs, compression={"UnitID":"SNAPPY","NumberofAlarms":"GZIP"})

TypeError: write_table() missing 1 required positional argument: 'where'

In [42]:
path1="s3a://pengfei/diffusion/data_format/arrow_snappy_fire_department.parquet"
path2="s3a://pengfei/diffusion/data_format/Fire_Department.parquet"
df=check_spark_parquet_read_time(path2)

data frame has 5500520 rows, 35 columns
Spark read above data frame in parquet format, and spents: 2.1240062713623047 s


In [20]:
print(df.columns)

NameError: name 'df' is not defined